In [1]:
import os
from pathlib import Path

import ee
import geemap
import geopandas as gpd
import shapely

In [2]:
ee.Initialize(project="ee-ursa-test")

In [3]:
data_path = Path(os.environ["DATA_PATH"])
ghsl_path = Path(os.environ["GHSL_PATH"])

In [4]:
bounds = gpd.read_file(
    data_path / "area_de_estudio" / "d_160101_ambito_estudio.shp",
).to_crs("EPSG:4326")

bbox = shapely.box(*bounds.total_bounds)
bbox_ee = ee.geometry.Geometry.Polygon(list(bbox.exterior.coords))

In [ ]:
col = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterBounds(bbox_ee)

# Full

In [10]:
img = col.filterDate("2020-01-01", "2020-12-31").mode().clip(bbox_ee)

geemap.download_ee_image(
    img, "./landuse.tif", scale=30, region=bbox_ee, crs="EPSG:4326",
)

landuse.tif: |          | 0.00/147M (raw) [  0.0%] in 00:00 (eta:     ?)

KeyboardInterrupt: 

In [ ]:
img_high = col.filterDate("2020-04-01", "2020-05-31").mode().clip(bbox_ee)

img_low = col.filterDate("2020-09-01", "2020-10-31").mode().clip(bbox_ee)

In [6]:
viz_params = {
    "bands": ["label"],
    "min": 0,
    "max": 8,
    "palette": [
        "#419bdf",
        "#397d49",
        "#88b053",
        "#7a87c6",
        "#e49635",
        "#dfc35a",
        "#c4281b",
        "#a59b8f",
        "#b39fe1",
    ],
}

m = geemap.Map(center=[-3.75, -73.25], zoom=10)
m.addLayer(img_high, viz_params, "High Season")
m.addLayer(img_low, viz_params, "Low Season")
m

Map(center=[-3.75, -73.25], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…